In [59]:
import tensorflow as tf
import cv2 as cv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import warnings
import re
warnings.filterwarnings("ignore")

In [60]:
from DataProcess import csv_process, load_img

In [80]:
class PillModel(tf.keras.Model):
    
    def __init__(self, class_num):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(96, (11, 11), strides=(4,4), padding='valid', activation='relu', input_shape=(227,227,3))
        self.conv2 = tf.keras.layers.Conv2D(256, (5, 5), strides=(1,1), padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(384, (3, 3), strides=(1,1), padding='same',activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(384, (3, 3), strides=(1,1), padding='same',activation='relu')
        self.conv5 = tf.keras.layers.Conv2D(256, (3, 3), strides=(1,1), padding='same',activation='relu')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.bn3 = tf.keras.layers.BatchNormalization()
        self.bn4 = tf.keras.layers.BatchNormalization()
        self.bn5 = tf.keras.layers.BatchNormalization()
        self.maxpool = tf.keras.layers.MaxPooling2D((3, 3), strides=(2,2))
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(4096, activation='relu')
        self.dense2 = tf.keras.layers.Dense(4096, activation='relu')
        self.dense3 = tf.keras.layers.Dense(class_num,activation='softmax')

    def call(self, input):
        x = self.conv1(input)
        x = self.bn1(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.maxpool(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.conv5(x)
        x = self.bn5(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        
        return x

In [62]:
path = '/content/drive/MyDrive/'

In [63]:
y1_train, y1_test, y1_val, my_dict = csv_process(path + 'merged_med_4.csv', '의약품제형')
y2_train, y2_test, y2_val, jh_dict = csv_process(path + 'merged_med_4.csv', '제형코드명')

In [64]:
X_train, X_val, X_test = load_img('/content/drive/MyDrive/cropped_pills')

In [81]:
model1 = PillModel(class_num=11)
model1.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            metrics=['accuracy'])

In [82]:
model1.build((None,227,227,3))

In [83]:
model1.summary()

Model: "pill_model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_63 (Conv2D)          multiple                  34944     
                                                                 
 conv2d_64 (Conv2D)          multiple                  614656    
                                                                 
 conv2d_65 (Conv2D)          multiple                  885120    
                                                                 
 conv2d_66 (Conv2D)          multiple                  1327488   
                                                                 
 conv2d_67 (Conv2D)          multiple                  884992    
                                                                 
 batch_normalization_22 (Bat  multiple                 384       
 chNormalization)                                                
                                                     

In [ ]:
history1 = model1.fit(X_train, y1_train, epochs=100, batch_size=32, validation_data=(X_val, y1_val))

In [ ]:
model1.predict(X_test,y1_test)

In [ ]:
model2 = PillModel(class_num=4)

In [ ]:
model2.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            metrics=['accuracy'])

In [ ]:
history2 = model2.fit(X_train, y2_train, epochs=100, batch_size=32, validation_data=(X_val, y1_val))

In [ ]:
model2.predict(X_test,y1_test)